In [3]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 1.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357284 sha256=4723a6c9f041e1693a6e35b6c6002dba22d8adc67f855282278b4cce3e8e76f5
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error

from surprise import Dataset, Reader
from surprise import KNNWithMeans, SVD, BaselineOnly, accuracy
from surprise.model_selection import train_test_split

import numpy as np

## Train

Logistic Regression

In [26]:
data_path = '/content/drive/MyDrive/CS 229 Group Project/Data/New/input_labels_df.csv'
data = pd.read_csv(data_path)

purchase_columns = [col for col in data.columns if col.startswith('y_verified_purchase')]
data[purchase_columns] = data[purchase_columns].astype(int)

user_features = [
    'user_average_rating', 'user_std_rating', 'avg_user_sentiment', 'user_review_count',
    'word_count', 'user_avg_word_count', 'helpfulness_ratio', 'verified_purchase_count',
    'user_avgrating_diff'
]

item_columns = [col for col in data.columns if col.startswith('avg_rating_')]

results = []
scaler = StandardScaler()

for item_rating_column, target_column in zip(item_columns, purchase_columns):
    X = data[user_features + [item_rating_column]]
    y = data[target_column]

    if len(y.unique()) < 2:
        print(f"Skipping {target_column} because it contains only one class.")
        continue

    X_scaled = scaler.fit_transform(X)

    log_reg = LogisticRegression(max_iter=1000, class_weight='balanced')
    log_reg.fit(X_scaled, y)

    y_pred_proba = log_reg.predict_proba(X_scaled)[:, 1]
    y_pred_log = (y_pred_proba >= 0.3).astype(int)

    accuracy = accuracy_score(y, y_pred_log)
    precision = precision_score(y, y_pred_log, zero_division=0)
    recall = recall_score(y, y_pred_log, zero_division=0)

    results.append({
        'Item': item_rating_column,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall
    })
results_df = pd.DataFrame(results)
print(results_df)

average_accuracy = results_df['Accuracy'].mean()
average_precision = results_df['Precision'].mean()
average_recall = results_df['Recall'].mean()

average_accuracy, average_precision, average_recall

rmse_accuracy = np.sqrt(mean_squared_error(results_df['Accuracy'], [average_accuracy] * len(results_df)))
rmse_precision = np.sqrt(mean_squared_error(results_df['Precision'], [average_precision] * len(results_df)))
rmse_recall = np.sqrt(mean_squared_error(results_df['Recall'], [average_recall] * len(results_df)))

rmse_accuracy, rmse_precision, rmse_recall

Skipping y_verified_purchase_B000067E30 because it contains only one class.
Skipping y_verified_purchase_B003Q3TRVQ because it contains only one class.
Skipping y_verified_purchase_B0046BPTI2 because it contains only one class.
Skipping y_verified_purchase_B004UL6O2I because it contains only one class.
Skipping y_verified_purchase_B005AL5H9S because it contains only one class.
Skipping y_verified_purchase_B006UQUSIW because it contains only one class.
Skipping y_verified_purchase_B007VXJK6E because it contains only one class.
Skipping y_verified_purchase_B007WFZ0XI because it contains only one class.
Skipping y_verified_purchase_B00AX3YNPC because it contains only one class.
Skipping y_verified_purchase_B00AYTMA00 because it contains only one class.
Skipping y_verified_purchase_B00FKSWNFM because it contains only one class.
Skipping y_verified_purchase_B00G7PCKI2 because it contains only one class.
Skipping y_verified_purchase_B00KCTER3U because it contains only one class.
Skipping y_v

KeyError: 'Accuracy'

In [30]:

average_accuracy = results_df['Accuracy'].mean()
average_precision = results_df['Precision'].mean()
average_recall = results_df['Recall'].mean()

print("Accuracy:", average_accuracy, "Precision:", average_precision, "Recall:", average_recall)

rmse_accuracy = np.sqrt(mean_squared_error(results_df['Accuracy'], [average_accuracy] * len(results_df)))
rmse_precision = np.sqrt(mean_squared_error(results_df['Precision'], [average_precision] * len(results_df)))
rmse_recall = np.sqrt(mean_squared_error(results_df['Recall'], [average_recall] * len(results_df)))

print("RMSE Accuracy:", rmse_accuracy, "RMSE Precision:", rmse_precision, "RMSE Recall:", rmse_recall)

Accuracy: 0.8912634408602151 Precision: 0.19825033399279543 Recall: 1.0
RMSE Accuracy: 0.11949741133497015 RMSE Precision: 0.25249627924569146 RMSE Recall: 0.0


In [14]:
train_file_path = '/content/drive/MyDrive/CS 229 Group Project/Data/users_with_100_reviews_df.csv'
test_file_path = "/content/drive/MyDrive/CS 229 Group Project/Data/New/user_top_1988_beauty_df.csv"


train_data = pd.read_csv(train_file_path)
test_data = pd.read_csv(test_file_path)

ratings_data = Dataset.load_from_df(train_data[['user_id', 'parent_asin', 'rating']], Reader(rating_scale=(0, 5)))
trainset = ratings_data.build_full_trainset()  # Build full trainset for Surprise models
testset = [(uid, iid, 0) for uid, iid in zip(test_data['user_id'], test_data['parent_asin'])]  # Testset with actual user-item pair

knn = KNNWithMeans(k=20, sim_options={'name': 'cosine', 'user_based': True})
knn.fit(trainset)
knn_predictions = knn.test(testset)
rmse_knn = accuracy.rmse(knn_predictions)

svd = SVD()
svd.fit(trainset)
svd_predictions = svd.test(testset)
rmse_svd = accuracy.rmse(svd_predictions)

# experimenting with BaselineOnly from the suprise package
# predicts reccomendations based on sum of biases (predicted_rating=global_average+user_bias+item_bias)
# bsl_options = {'method': 'als', 'n_epochs': 10, 'reg_u': 15, 'reg_i': 10}
# baseline_algo = BaselineOnly(bsl_options=bsl_options)
# baseline_algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 4.2348
RMSE: 4.2089


## Test

In [15]:
# logreg
# y_pred_logreg = log_reg.predict(X_test)

#kNN
knn_predictions = knn.test(testset)
rmse_knn = accuracy.rmse(knn_predictions)

#SVD
svd = SVD()
svd_predictions = svd.test(testset)

#baselineOnly
# baseline_predictions = baseline_algo.test(testset)

RMSE: 4.2348


AttributeError: 'SVD' object has no attribute 'trainset'

## Evaluate
Using RMSE

In [ ]:
# log reg
rmse_logreg = np.sqrt(mean_squared_error(y_test, y_pred_logreg))
print("Logistic Regression RMSE:", rmse_logreg)

#knn
rmse_knn = accuracy.rmse(knn_predictions)
print("KNNWithMeans RMSE:", rmse_knn)


#SVD
rmse_svd = accuracy.rmse(svd_predictions)
print("SVD RMSE:", rmse_svd)

# baselineOnly
rmse_baseline = accuracy.rmse(baseline_predictions)
print("BaselineOnly RMSE:", rmse_baseline)